<a href="https://colab.research.google.com/github/HaitaanshDixit/Research_Paper/blob/main/BLIP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate timm
!pip install git+https://github.com/salesforce/BLIP.git

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

In [ ]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import files
from PIL import Image

# Upload image
uploaded = files.upload()

# Open the first uploaded image
image_path = list(uploaded.keys())[0]
image = Image.open(image_path).convert("RGB")


In [ ]:
text = """A house is being manifested by a young adult who tries to give his best day to day everyday."""
instruction = f"Read the text and image and tell what is happening\n{text_input}"

In [ ]:
inputs = processor(images=image, text= instruction+text, return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.96,
    top_p=0.9,
    repetition_penalty=1.2,
)

summary = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Summary:\n", summary)